# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import nltk
#nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import pickle
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import nltk
from nltk.corpus import stopwords

[nltk_data] <urlopen error [Errno 8] nodename nor servname provided,
[nltk_data]     or not known>


In [2]:
print(os.getcwd())

/Users/davideffiong/Documents/Disaster_Response_Pipepline


In [3]:

# load data from database
#def load_data():
engine = create_engine('sqlite:////Users/davideffiong/Documents/Disaster_Response_Pipepline/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM disaster_table", engine)
X = df['message']
Y = df.iloc[:,4:]
#    return X, Y

load_data()

### 2. Write a tokenization function to process your text data

In [4]:
print(X[0])

Weather update - a cold front from Cuba that could pass over Haiti


In [5]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
lemmatizer

<WordNetLemmatizer>

In [6]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [25]:
print(tokenize(X[0]))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
df.shape

(26216, 40)

In [29]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9732bdc680>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [12]:
Y_preds = pipeline.predict(X_test)

In [42]:
results_dict = {}

for pred, label, col in zip(Y_preds.transpose(), Y_test.values.transpose(), Y_test.columns):
    print(col)
    print(classification_report(label, pred))
    results_dict[col] = classification_report(label, pred, output_dict=True)

related
              precision    recall  f1-score   support

           0       0.22      0.13      0.16      1759
           1       0.76      0.85      0.80      6040
           2       0.00      0.00      0.00        66

    accuracy                           0.68      7865
   macro avg       0.33      0.33      0.32      7865
weighted avg       0.64      0.68      0.65      7865

request
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      6497
           1       0.17      0.10      0.12      1368

    accuracy                           0.76      7865
   macro avg       0.50      0.50      0.49      7865
weighted avg       0.71      0.76      0.73      7865

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      786

/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865

    accuracy                           1.00      7865
   macro avg       1.00      1.00      1.00      7865
weighted avg       1.00      1.00      1.00      7865

water
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7324
           1       0.04      0.01      0.02       541

    accuracy                           0.91      7865
   macro avg       0.48      0.49      0.49      7865
weighted avg       0.87      0.91      0.89      7865

food
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      6940
           1       0.12      0.09      0.10       925

    accuracy                           0.82      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.79      0.82      0.81      7865

shelter
              precision    recall  f1-score   support


/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/david_data/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

earthquake
              precision    recall  f1-score   support

           0       0.91      0.92      0.91      7150
           1       0.11      0.10      0.10       715

    accuracy                           0.84      7865
   macro avg       0.51      0.51      0.51      7865
weighted avg       0.84      0.84      0.84      7865

cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7676
           1       0.00      0.00      0.00       189

    accuracy                           0.97      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.95      0.97      0.96      7865

other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7444
           1       0.06      0.00      0.00       421

    accuracy                           0.94      7865
   macro avg       0.50      0.50      0.49      7865
weighted avg       0.90      0.94      0.92 

In [31]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9732bdc680>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f9732bdc680>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [43]:
weighted_avg = {}
for key in results_dict.keys():
    weighted_avg[key] = results_dict[key]['weighted avg']

df_w = pd.DataFrame(weighted_avg).transpose()
df_w

,precision,recall,f1-score,support
related,0.635458,0.679339,0.653487,7865.0
request,0.710859,0.758296,0.731757,7865.0
offer,0.990866,0.995423,0.993139,7865.0
aid_related,0.513093,0.520534,0.515676,7865.0
medical_help,0.860988,0.914050,0.880100,7865.0
medical_products,0.902148,0.944056,0.922178,7865.0
search_and_rescue,0.947754,0.971519,0.959490,7865.0
security,0.968214,0.983853,0.975971,7865.0
military,0.939701,0.965798,0.952096,7865.0
child_alone,1.000000,1.000000,1.000000,7865.0


In [45]:
def model_performance(labels, preds):
    # Print/collect results
    results_dict = {}
    for pred, label, col in zip(preds.transpose(), labels.values.transpose(), labels.columns):
        print(col)
        print(classification_report(label, pred))
        results_dict[col] = classification_report(label, pred, output_dict=True)
    
    # Convert to df
    weighted_avg = {}
    for key in results_dict.keys():
        weighted_avg[key] = results_dict[key]['weighted avg']

    df_w = pd.DataFrame(weighted_avg).transpose()
    
    # Show metrics for "key categories"
    key_categories = ['related', 'request', 'aid_related', 'medical_help',
                      'shelter', 'other_aid', 'infrastructure_related',
                      'weather_related', 'direct_report']
    
    print(df_w.loc[key_categories]['f1-score'].describe())
    
    return df_w

### 6. Improve your model
Use grid search to find better parameters. 

In [46]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9732bdc680>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f9732bdc680>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [48]:
#make_scorer(f1_score, average='micro')

parameters = {
    'text__ngram_range': ((1, 1), (1, 2)),
    'text__max_df': [0.5],
    'text__max_features': [5000],
    'text__use_idf': [False],
    'clf__estimator__max_depth': (25, 50, 100, None),
    'clf__estimator__min_samples_split': (2, 10, 25, 50, 100), 
    'clf__estimator__n_estimators': [200]
}

cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=3)


In [49]:
cv

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f9732bdc680>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=3,
             param_grid={'clf__estimator__max_depth': (25, 50, 100, None),
                         'clf__estimator__min_samples_split': (2, 10, 25, 50,
                                                               100),
                         'clf__estimator__n_estimators': [200],
                         'text__max_df': [0.5], 'text__max_features': [5000],
                         'text__ngram_range': ((1, 1), (1, 2)),
                         'text__use_idf': [False]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [50]:

cv.fit(X_train, Y_train)

ValueError: Invalid parameter text for estimator Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8804bc8200>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [17]:
def display_results(cv, Y_test, Y_preds):
    labels = np.unique(Y_preds)
    confusion_mat = confusion_matrix(Y_test, Y_preds, labels=labels)
    accuracy = (Y_pred == Y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)


def main():
#    X, Y = load_data()
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

    #model = build_model()
    cv.fit(X_train, Y_train)
    Y_pred = cv.predict(X_test)

    display_results(model, Y_test, Y_pred)

main()

ValueError: Invalid parameter min_samples_split for estimator MultiOutputClassifier(estimator=RandomForestClassifier()). Check the list of available parameters with `estimator.get_params().keys()`.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.